<a href="https://colab.research.google.com/github/antonellagambarte/procesamiento_del_lenguaje_I/blob/main/desafio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
%pip install numpy scikit-learn

In [20]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups
import numpy as np

In [21]:
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

In [25]:

tfidfvect = TfidfVectorizer()
X_train = tfidfvect.fit_transform(newsgroups_train.data)

In [26]:
np.random.seed(42)
idxs = np.random.choice(len(newsgroups_train.data), 5, replace=False)

In [27]:
for i, idx in enumerate(idxs):
    print("="*80)
    print(f"Documento {i+1} (índice {idx}) - Clase: {newsgroups_train.target_names[newsgroups_train.target[idx]]}")
    print(newsgroups_train.data[idx][:800], "...\n")
    cossim = cosine_similarity(X_train[idx], X_train)[0]
    most_similar = np.argsort(cossim)[::-1][1:6]
    print("5 documentos más similares:")
    for j in most_similar:
        print(f" - idx {j}, clase: {newsgroups_train.target_names[newsgroups_train.target[j]]}, similitud: {cossim[j]:.3f}")
    print()

Documento 1 (índice 7492) - Clase: comp.sys.mac.hardware
Could someone please post any info on these systems.

Thanks.
BoB
-- 
---------------------------------------------------------------------- 
Robert Novitskey | "Pursuing women is similar to banging one's head
rrn@po.cwru.edu  |  against a wall...with less opportunity for reward"  ...

5 documentos más similares:
 - idx 10935, clase: comp.sys.mac.hardware, similitud: 0.667
 - idx 7258, clase: comp.sys.ibm.pc.hardware, similitud: 0.348
 - idx 4971, clase: comp.sys.mac.hardware, similitud: 0.180
 - idx 4303, clase: misc.forsale, similitud: 0.155
 - idx 645, clase: comp.sys.mac.hardware, similitud: 0.141

Documento 2 (índice 3546) - Clase: comp.os.ms-windows.misc


     Don't bother if you have CPBackup or Fastback.  They all offer options 
not available in the stripped-down MS version (FROM CPS!).  Examples - no 
proprietary format (to save space), probably no direct DMA access, and no 
tape drive! ...

5 documentos más similares:
